In [1]:
!pip install --quiet -U ipykernel
!pip install --quiet -U langchain
!pip install --quiet -U langchain-openai
!pip install --quiet -U langgraph
!pip install --quiet -U langchainhub
!pip install --quiet -U tavily-python
!pip install --quiet -U langchain-community
!pip install --quiet -U python-dotenv==1.0.1
!pip install --quiet -U langchain-anthropic
!pip install --quiet -U mlflow
!pip install --quiet -U openai

In [2]:
import os
from typing import Dict, List, Optional, Any, Literal

# Tavily API
from tavily import TavilyClient

# Langchain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_classic.vectorstores import Chroma
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# Langgraph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.state import CompiledStateGraph

# Evaluation 
import mlflow

# Anthropic
from anthropic import Anthropic

# Environment Variables
from dotenv import load_dotenv
from IPython.display import Image, display

/Users/rmiles/Desktop/local_projects/Python_for_AI/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:
load_dotenv("config.env")

base_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [4]:
tavily_client = TavilyClient()

## Create our tool node and LLM with Tools

In [5]:
# Lets define our tavily search tool node

@tool

# Here we define our function with an input "question" and hint that its a string. the operator "->" provides a hint that the return type is a Dict and should be JSON-like
def web_search(question:str)->Dict:
    """
    Return top search results for a given search query.
    """
    # Below we call Tavily's software development kit's (SDK) .search() and provide the input given the the function which should be a user inputted string.
        # This performs a synchronous HTTPS request using the API key bound to my 'tavily_client' that i have instantiated already
        # the above call to tavily's SDK and its function to perform the HTTPS request with the API key gets stored in the 'response' object.
    response = tavily_client.search(question)
    print("im at the tool node")
    # below we return our object which should be a Dict. 
    # In an agent run, LangChain serializes this to JSON and passes it back to the model as a ToolMessage so the model can read and use the results for the next reasoning step.
    return response

In [6]:
tools = [web_search]

In [7]:
llm_with_tools = base_llm.bind_tools(tools)

## Create our State Schema

In [ ]:
class State(MessagesState):
    topic: Optional[str]
    summary: Optional[str]
    quiz_question: Optional[str]
    quiz_options: Optional[List[str]]
    quiz_correct_option: Optional[str]
    patient_answer: Optional[str]
    evaluation: Optional[Dict[str, Any]]
    phase: Optional[
        Literal[
            "ask_topic",
            "searching",
            "show_summary",
            "waiting_ready",
            "quiz_generated",
            "waiting_answer",
            "evaluated",
            "waiting_restart"
        ]
    ]
    repeat_mode: bool
    #Dont need a messages variable because the MessagesState autoimatically includes a messages field

## Create an entrypoint node. 

This node should also be the introduction of the system to the user. 

This node will have an interrupt after to collect the topic the user wants to learn about. 

In [ ]:
# lets create our entrypoint into our langgraph workflow.

def entrypoint(state: State):
    sys_message = SystemMessage(
        content=(
            "You are a helpful assistant that conducts web searches to help patients understand their medical conditions " \
            "treatment options, and post-treatment care. " \
            "Your job is to answer patient querieswith web searches, if needed, and provide summarized medical information. " \
            "After. you provide information to the user, you will also quiz the user."))
    ai_message = AIMessage(
        content=(
            "What health topic or medical condition do you want to learn about?"
        )
    )
    state["messages"] = state["messages"] + [sys_message] + [ai_message]
    return {"messages": [sys_message, ai_message], 
            "phase": "ask_topic",}



# Need an interrupt after this node to be able to store my topic. This interrupt will be an input for the user to be able to input their desired topic they want to learn about.

## Create our Agent node

In [ ]:
# Lets create our llm agent node

def agent(state: State):
    topic = state["topic"]
    messages = state["messages"]
    if topic is None:
        messages = messages + [
            HumanMessage(
                content=(
                    f"Please search the web for up-to-date patient-friendly information about {topic}. "
                    "Summarize what the topic is for the patient, provide key points, and general treatment/management options, in simple english." \
                    "This is for education only, not medical advice."
                )
            )
        ]
        

    ai_message = llm_with_tools.invoke(messages)
    return {"messages": messages + [ai_message], "phase": "searching",}

In [ ]:
def summarizer(state: State):
    messages = state["messages"]
    summary = state["messages"][-1]
    for m in summary:
        m.pretty_print()
    return {"messages": messages + [summary], "phase": "show_summary",}

In [ ]:
def quiz_node(state: State):
    messages = state["mesages"]
    sys_message = SystemMessage(
        content=(
            "You are an assistant that quizzes patients with medical conditions. Your job is to look at the previous information shared with the patient and generate a quiz for them, helping to reinforce their understanding of their condition."))
    ai_message = AIMessage(
        content=(
            "Are you ready to take a quiz on the information you were just provided?"))
    state["messages"] + [sys_message + ai_message]
    quiz_question = input("Yes or no: ")
    if quiz_question.lower() == "yes":
        quiz = base_llm.invoke(messages)
        unformatted_quiz = quiz.content
        for m in unformatted_quiz:
            m.pretty_print()
    else:
        return "END"
    return {"messages": messages + [sys_message, ai_message]}

In [ ]:
def router1(state: State):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tool_node"
    return "summarizer"

In [ ]:
def router2(state: State):
    last_message = state["messages"][-1]
    if last_message

In [ ]:
# Create our graph workflow

workflow = StateGraph(State)

#workflow.add_node("entrypoint", entrypoint)
workflow.add_node("agent", agent)
workflow.add_node("tool_node", ToolNode([web_search]))
workflow.add_node("summarizer", summarizer)
workflow.add_node("quiz_node", quiz_node)
##workflow.add_node("node_2", ToolNode([web_search]))
#workflow.add_node("node_3", )
#workflow.add_node("node_4", )
#workflow.add_node("node_5", )

workflow.add_edge(START, "agent")
#workflow.add_edge("agent", "first_agent")

workflow.add_conditional_edges(
    source= "agent",
    path=router1,
    path_map= ["tool_node", "summarizer"]
)

#workflow.add_conditional_edges(
    #source="presenter_node",
   #path=continue_router,
    #path_map=["entrypoint", "END"]
#)

workflow.add_edge("tool_node", "agent")
workflow.add_edge("summarizer", "quiz_node")
workflow.add_edge("quiz_node", END)

In [ ]:

graph = workflow.compile()
#memory = MemorySaver()
#graph = workflow.compile(
#    interrupt_after=["entrypoint"],
#    checkpointer=memory
#)

In [ ]:
display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
def human_in_loop_ask_topic(graph: CompiledStateGraph, question:str, thread_id:int):
    topic = {"topic": topic}
    #config = {"configurable": {"thread_id": thread_id}}
    for event in graph.stream(input=topic, stream_mode="values"):
        if not event['messages']:
            continue
        event['messages'][-1].pretty_print()

    human_input = input("Do you approve the tool calling(Y or N)?: ")
        
    if human_input.lower() == "yes":
        for event in graph.stream(input=None, stream_mode="values"):
            if not event['messages']:
                continue
            event['messages'][-1].pretty_print()
    else:
        AIMessage("Please provide a topic that you want to learn about.").pretty_print()


In [ ]:
test = graph.invoke(
    {"topic": "What is COPD?"}
)

print(test)

In [ ]:
for message in test["messages"]:
    message.pretty_print()